In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine
from fsUtils import moveFile
from ioUtils import getFile
from timeUtils import clock, elapsed
from pandas import DataFrame, Series, read_csv


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-11-01 10:32:16.116339


In [118]:
oscarsURL = "https://www.imdb.com/oscars/nominations/?ref_=ev_all"
baftasURL = "https://www.imdb.com/awards-central/baftas/?ref_=ev_all"
goldenglobesURL = "https://www.imdb.com/golden-globes/nominations/?ref_=ev_all"
sagURL = "https://www.imdb.com/event/ev0000598/2019/1"
cannesURL = "https://www.imdb.com/event/ev0000147/2019/1"
bodilURL = "https://www.imdb.com/event/ev0000107/2019/1"
torontoURL = "https://www.imdb.com/event/ev0000659/2019/1"
sundanceURL = "https://www.imdb.com/event/ev0000631/2019/1"
soundURL = "https://www.imdb.com/event/ev0000452/2019/1"
teenURL = "https://www.imdb.com/event/ev0000644/2019/1"

In [119]:
from webUtils import getWebData

url      = oscarsURL
savename = "awards/IMDB_oscars_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = baftasURL
savename = "awards/IMDB_baftas_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = goldenglobesURL
savename = "awards/IMDB_goldenglobes_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = sagURL
savename = "awards/IMDB_sag_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = cannesURL
savename = "awards/IMDB_cannes_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = bodilURL
savename = "awards/IMDB_bodil_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = torontoURL
savename = "awards/IMDB_toronto_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = sundanceURL
savename = "awards/IMDB_sundance_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = soundURL
savename = "awards/IMDB_sound_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = teenURL
savename = "awards/IMDB_teen_base.p"
getWebData(base=url, savename=savename, useSafari=False, debug=True)

  --> This file is 44.9kB.
awards/IMDB_teen_base.p size -> 45 kB


True

# Parse Data

In [102]:
#from webUtils import getHTML
#oscarsData = getHTML("awards/IMDB_oscars_base.p")

In [144]:
from time import sleep
import json
from searchUtils import findPatternExt


movieMap = {"baftas": "BAFTA Film Award", "oscars": "Oscars", "sag": "Actor", "teen": "Teen Choice Award", "sound": "Golden Reel Award", "toronto": "People's Choice Award", "sundance": "Grand Jury Prize", "cannes": "Palme d'Or", "bodil": "Bodil"}


def getPageData(data, awardsName):
    dd = data["nomineesWidgetModel"]['eventEditionSummary']
    awards = dd['awards']
    year   = dd['year']
    
    dataname = movieMap.get(awardsName)
    
    awardsData = {}
    for award in awards:
        if award['awardName'] != dataname:
            continue
        for category in award['categories']:
            name     = category["categoryName"]
            nominees = category["nominations"]
            categoryData = {"Winner": [], "Nominees": []}
            for nominee in nominees:
                primary   = nominee['primaryNominees']
                secondary = nominee['secondaryNominees']
                isWinner  = nominee['isWinner']

                primaryData   = [x['name'] for x in primary]
                secondaryData = [x['name'] for x in secondary]
                catData = {"Primary": primaryData, "Secondary": secondaryData}
                if isWinner is True:
                    categoryData["Winner"].append(catData)
                else:
                    categoryData["Nominees"].append(catData)

            awardsData[name] = categoryData
            
    retval = {year: awardsData}
    return retval

def parseOscarsData(awardsName="oscars"):

    awardsData = {}
    files = findPatternExt("awards", pattern="IMDB_{0}_".format(awardsName), ext=".p")
    #print("Found {0} files".format(len(files)))
    for oscarsFile in files:
        if "base.p" in oscarsFile:
            continue
            
        oscarsData = getHTML(oscarsFile)
        for sdata in oscarsData.findAll("script", {"type": "text/javascript"}):

            txt = sdata.text
            pos = txt.find("IMDbReactWidgets.NomineesWidget.push")
            if pos == -1:
                continue
            start = pos + len("IMDbReactWidgets.NomineesWidget.push(")
            end   = txt.rfind("]);") + 1

            txtdata = txt[start:end]
            try:
                data = json.loads(txtdata[len("['center-8-react',"):-1])
            except:
                raise ValueError("Cannot find dictionary in data!!!")

            retval = getPageData(data, awardsName)
            awardsData.update(retval)
    return awardsData
    
    
def getOscarsData(awardsName="oscars"):
    awardsData = {}
    
    oscarsData = getHTML("awards/IMDB_{0}_base.p".format(awardsName))
    for sdata in oscarsData.findAll("script", {"type": "text/javascript"}):

        txt = sdata.text
        pos = txt.find("IMDbReactWidgets.EventHistoryWidget.push")
        if pos == -1:
            continue
        start = pos + len("IMDbReactWidgets.EventHistoryWidget.push(")
        end   = txt.rfind("]);") + 1

        txtdata = txt[start:end]
        try:
            data = json.loads(txtdata[len("['right-1-react',"):-1])
        except:
            raise ValueError("Cannot find dictionary in data!!!")

        eventID = data['eventHistoryWidgetModel']['eventId']
        for event in data['eventHistoryWidgetModel']['eventEditions']:
            #eventID  = event['eventEditionId']
            year     = event['year']

            #url      = "https://www.imdb.com/event/ev0000003/2018/1/
            url      = "https://www.imdb.com/event/{0}/{1}/1/".format(eventID, year)
            savename = "awards/IMDB_{0}_{1}.p".format(awardsName, year)
            try:
                getWebData(base=url, savename=savename, useSafari=False, debug=True)        
                sleep(1.5)
            except:
                break
    
    #retval = getPageData(data)
    #awardsData.update(retval)
    #print(len(awardsData))
    
    print("\n\n\n")

In [142]:
#getOscarsData("teen")
#parseOscarsData()

In [147]:
eventData = {}
for event in movieMap.keys():
    eventData[event] = parseOscarsData(event)

In [148]:
from ioUtils import getFile, saveFile

In [ ]:
saveFile(idata=eventData, ifile="awards")